In [443]:
import json
import re
from spacy.lang.en import English # updated

In [262]:
tagged_conv = []
taggs = []
tagged = ["tc_processed/train_full_anno.json", "tc_processed/test_freq_full_anno.json", 
          "tc_processed/test_rare_full_anno.json", "tc_processed/valid_freq_full_anno.json", 
          "tc_processed/valid_rare_full_anno.json"]
for each in tagged:
    with open(each) as f:
        data = json.load(f)
    keys = list(data.keys())
    for each in keys:
        content = data[each]['content']
        c, t = get_conversation(content)
        tagged_conv.append(c)
        taggs.append(t)

In [274]:
non = "processed_output/train.src"
with open(non) as f:
    src = f.readlines()
non_tgt = "processed_output/train.tgt"
with open(non_tgt) as f:
    tgt = f.readlines()

In [280]:
original = []
for i in range(len(src)-1):
    if src[i].replace("\n", "") not in src[i+1]:
        original.append(src[i])

In [429]:
for i in range(len(src)):
    text = src[i]
    text = text.replace("_eos", "")
    text = text.replace("\n", "")
    text = text.replace(" ", "")
    text = re.sub('\W+', '', text)
    src[i] = text
conversations = []
for i in range(len(src)-1):
    if src[i] not in src[i+1]:
        ending = tgt[i].replace("\n", "")
        ending = ending.replace(" ", "")
        ending = ending.replace("_eos", "")
        ending = ending.replace("_go", "")
        ending = re.sub('\W+', '', ending)
        c = src[i] + ending
        conversations.append(c)
ending = tgt[len(src)-1].replace("\n", "")
ending = ending.replace(" ", "")
ending = ending.replace("_eos", "")
ending = ending.replace("_go", "")
ending = re.sub('\W+', '', ending)
c = src[len(src)-1] + ending
conversations.append(c)

In [338]:
not_in_convs = []
for each in not_in:
    current_idx = src.index(original[each])
    previous = src[current_idx - 1].replace("\n", "")
    current = src[current_idx].replace("\n", "")
    c = [current]
    while previous in current:
        c.insert(0, previous)
        current_idx -= 1
        current = previous
        previous = src[current_idx - 1].replace("\n", "")
    not_in_convs.append(c)
    

In [452]:
not_in_convs[1]

['yes i like very much _eos',
 'yes i like very much _eos the earliest western asian epic poetry , the epic of gilgamesh , was written in sumerian _eos',
 'yes i like very much _eos the earliest western asian epic poetry , the epic of gilgamesh , was written in sumerian _eos edgar allan poe only received $ 9 for the publication of his poem , " the raven . " _eos',
 'yes i like very much _eos the earliest western asian epic poetry , the epic of gilgamesh , was written in sumerian _eos edgar allan poe only received $ 9 for the publication of his poem , " the raven . " _eos how much do you think the original manuscript would be worth now ? _eos',
 'yes i like very much _eos the earliest western asian epic poetry , the epic of gilgamesh , was written in sumerian _eos edgar allan poe only received $ 9 for the publication of his poem , " the raven . " _eos how much do you think the original manuscript would be worth now ? _eos sorry i do n\'t know _eos',
 'yes i like very much _eos the earli

In [451]:
for each in not_in_convs:
    print(len(each))

18
11
20
20
21
20


In [450]:
for each in not_in_convs:
    print(each[0])

hi , how are you ? _eos
yes i like very much _eos
do you follow elections closely _eos
the green bay packers are my favorite football team . i was bummed over the loss to the redskins though ! _eos
hello , do you pay much attention to what 's going on in the media ? _eos
are you a voter ? _eos


In [337]:
src.index(original[2066])

43219

In [334]:
not_in

[2066, 3475, 4656, 5664, 6348, 7898]

In [261]:
def get_conversation(content):
    messages = []
    tags = []
    for turn in content:
        messages.append(turn['message'])
        tags.append(turn['mezza_da'])
    messages = "".join(messages)
    messages = messages.replace(" ", "").lower()
    messages = messages.replace("\n", "")
    messages = messages.replace("\t", "")
    messages = re.sub('\W+','', messages)
    return "".join(messages), tags

In [430]:
ids = []
for i in range(len(conversations)):
    if conversations[i] in tagged_conv:
        ids.append(tagged_conv.index(conversations[i]))
    else:
        ids.append("NA")

In [438]:
output = []
na_count = 0
untagged_len = []
tagged_len = []
for conv in ids:
    if conv != "NA":
        untagged_len.append(len(taggs[conv])-1)
        conv_tag = taggs[conv]
        for i in range(1, len(conv_tag)):
            output.append(get_taggs(conv_tag[:i]))
    else:
        untagged_len.append(len(not_in_convs[na_count]))
        output = output + not_in_convs[na_count]
        na_count +=1 


In [447]:
with open('processed_output/train.src.da', 'w') as filehandle:
    filehandle.writelines("%s\n" % place for place in output)

In [445]:
nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer'))

In [439]:
for i in range(len(output)):
    docs = nlp(output[i])
    

179709

In [385]:
length_of_each_conv = []
count = 1
for i in range(len(src)-1):
    if src[i].replace("\n", "") not in src[i+1]:
        length_of_each_conv.append(count)
        count = 1
    else:
        count += 1
length_of_each_conv.append(count)

In [386]:
sum(length_of_each_conv)

179709

In [345]:
def get_taggs(tag_list):
    output = ""
    for i in range(len(tag_list)):
        for each in tag_list[i]:
            output += f"{each['da']} "
        if i != len(tag_list) - 1:
            output += "_eos "
    return output[:-1]
        
        

In [301]:
not_in

[2066, 3475, 4656, 5664, 6348, 7898]

In [317]:
for i in range(len(tagged_conv)):
    if '1988' in tagged_conv[i] and '75' in tagged_conv[i]:
        if '400000' in tagged_conv[i]:
            print(i, tagged_conv[i])

3925 hihowareyoufineijustreturnedfromatripitraveledtoeuropeiseemslikeairtravelisslowerthesedaysthatitusedtobeamirightyeahsomanysecuritychecksitsreallyirritatingiwouldlovefasterplansthatcouldgoover1000mphorahighspeedrailserviceintheusforintercitytravelthedutcharedesigningaspeedybusforcommuterswhycantwerightyeahourbussystmsaresoslowthedriversroutinelyreleaserabbitsatthebusstopsothatfuturepassengershavesomethingtoeatwhiletheyarewaitingforthenextbuskindoflikeshipsusedtodoleavingrabbitsonislandsilikethatthoughtfulofthedriversbutprobablyincreasesbusfaresifkingkalakauaofhawaiihadtravelledtheworldonbushewouldhavediedbeforecompletingthejourneyyeahhecouldhavewalkedfasterkindofliketheairlineswhoinsteadoffetchingbaggagefasterjustmakeuswalkfurtheriheardtheydidthatinhoustonallthatcomplaintsaboutwaitingforbagsheywhydontyoutakealongwalkaroundtheairportbeforeyougetyourbagsthenthepassengersdidntgetavoteinthatdecisionallthosepoorpeoplecarryingaroundtheirstolengardengnomesatleastitincreasestimefortakingse